In [ ]:
import numpy as np
import pandas as pd
from sklearn import  preprocessing, ensemble
from sklearn.metrics import log_loss,accuracy_score
from sklearn.cross_validation import KFold,StratifiedKFold
import re
import string
from collections import defaultdict, Counter
#import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
from mochi import *

In [ ]:
#models to be used
import xgboost as xgb


In [ ]:
KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 42)


In [ ]:
#basic feature generation
#some transfromed features
train_df["num_photos"] = train_df["photos"].apply(len)
train_df["num_features"] = train_df["features"].apply(len)
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))

#time-related
train_df["created"] = pd.to_datetime(train_df["created"])

train_df["created_year"] = train_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour

train_df["dayofyear"] = train_df["created"].dt.dayofyear

train_df["sine_hour"] = np.sin(2*np.pi*train_df["created_hour"]/24)
train_df["cos_hour"] = np.cos(2*np.pi*train_df["created_hour"]/24)

train_df["sine_weekday"] = np.sin(2*np.pi*train_df["created_weekday"]/7)
train_df["cos_weekday"] = np.cos(2*np.pi*train_df["created_weekday"]/7)

train_df["sine_day"] = np.sin(2*np.pi*train_df["created_day"]/7)
train_df["cos_day"] = np.cos(2*np.pi*train_df["created_day"]/7)

#some new numerical features related to the price
train_df["price_per_bath"] =  (train_df["price"]*1.0/train_df["bathrooms"]).replace(np.Inf,np.nan)
train_df["price_per_bed"] = (train_df["price"]*1.0/train_df["bedrooms"]).replace(np.Inf,np.nan)
train_df["bath_per_bed"] = (train_df["bathrooms"]*1.0/train_df["bedrooms"]).replace(np.Inf,np.nan)
train_df["price_per_room"] = (train_df["price"]*1.0/(train_df["bedrooms"]+train_df["bathrooms"])).replace(np.Inf,np.nan)

train_df['house_type']=map(lambda x,y:(x,y),train_df['bedrooms'],train_df['bathrooms'])
train_df['house_type'] = train_df['house_type'].apply(str)

ny_lat = 40.785091
ny_lon = -73.968285
train_df['central_distance']= np.sqrt((train_df['latitude']-ny_lat)**2 + (train_df['longitude']-ny_lon)**2) 


In [ ]:
#from non-structured features
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_features"] = test_df["features"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

#time-related
test_df["created"] = pd.to_datetime(test_df["created"])

test_df["created_year"] = test_df["created"].dt.year
test_df["created_month"] = test_df["created"].dt.month
test_df["created_day"] = test_df["created"].dt.day
test_df["created_hour"] = test_df["created"].dt.hour

test_df["dayofyear"] = test_df["created"].dt.dayofyear

test_df["sine_hour"] = np.sin(2*np.pi*test_df["created_hour"]/24)
test_df["cos_hour"] = np.cos(2*np.pi*test_df["created_hour"]/24)

test_df["sine_weekday"] = np.sin(2*np.pi*test_df["created_weekday"]/7)
test_df["cos_weekday"] = np.cos(2*np.pi*test_df["created_weekday"]/7)

test_df["sine_day"] = np.sin(2*np.pi*test_df["created_day"]/7)
test_df["cos_day"] = np.cos(2*np.pi*test_df["created_day"]/7)

#some new numerical features related to the price

test_df["price_per_bath"] =  (test_df["price"]*1.0/test_df["bathrooms"]).replace(np.Inf,np.nan)
test_df["price_per_bed"] = (test_df["price"]*1.0/test_df["bedrooms"]).replace(np.Inf,np.nan)
test_df["bath_per_bed"] = (test_df["bathrooms"]*1.0/test_df["bedrooms"]).replace(np.Inf,np.nan)
test_df["price_per_room"] = (test_df["price"]*1.0/(test_df["bedrooms"]+test_df["bathrooms"])).replace(np.Inf,np.nan)

test_df['house_type']=map(lambda x,y:(x,y),test_df['bedrooms'],test_df['bathrooms'])
test_df['house_type'] = test_df['house_type'].apply(str)

test_df['central_distance']= np.sqrt((test_df['latitude']-ny_lat)**2 + (test_df['longitude']-ny_lon)**2) 

In [ ]:
"""
new categorical data generated from the old ones
"""
#new feature for the street_address, use them instead of the original one
train_df["street_name"] = train_df["street_address"].apply(proecessStreet)
test_df["street_name"] = test_df["street_address"].apply(proecessStreet)

train_df['building0']=map(lambda x:1 if x== '0' else 0,train_df['building_id'])
test_df['building0']=map(lambda x:1 if x== '0' else 0,test_df['building_id'])

In [ ]:
#dealing with features
#preprocessing for features
train_df["features"] = train_df["features"].apply(lambda x:["_".join(i.split(" ")).lower().strip().replace('-','_') \
                                                            for i in x])
test_df["features"] = test_df["features"].apply(lambda x:["_".join(i.split(" ")).lower().strip().replace('-','_')\
                                                          for i in x])
#create the accept list
accept_list = list(featureList(train_df,test_df,limit = 0.001))

#map the feature to dummy slots
featureMapping(train_df,test_df,accept_list)
features_to_use.extend(map(lambda x : 'with_'+x,accept_list))

In [ ]:
#generating the statistical features
